In [1]:
# 데이터 전처리
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import torch
import os

In [ ]:
# --- 경로 설정 ---
data_dir = 'safebooru\data'
processed_data_path = os.path.join(data_dir, 'processed_data.csv')

In [ ]:
# --- 설정 ---
TAG_FREQ_THRESHOLD = 1000 # 이 빈도수 이상의 태그만 사용

In [4]:
df = pd.read_csv(processed_data_path)

In [5]:
# 태그를 리스트로 변환 및 'tagme' 제거
df['tags'] = df['tags'].astype(str).apply(lambda x: [t for t in x.split() if t != 'tagme'])

# 태그 빈도 계산 및 필터링
tag_counts = pd.Series([tag for tags in df['tags'] for tag in tags]).value_counts()
valid_tags = tag_counts[tag_counts >= TAG_FREQ_THRESHOLD].index
df['tags'] = df['tags'].apply(lambda tags: [t for t in tags if t in valid_tags])

# 태그 이진화 (One-Hot Encoding)
mlb = MultiLabelBinarizer(classes=valid_tags)
tags_encoded = mlb.fit_transform(df['tags'])
num_tags = len(mlb.classes_)

# 이진화된 태그를 데이터프레임에 추가
df_tags = pd.DataFrame(tags_encoded, columns=mlb.classes_, index=df.index)
df = pd.concat([df.drop(columns=['tags']), df_tags], axis=1)

print(f"✅ 태그 필터링 및 이진화 완료. 최종 태그 수: {num_tags}")

✅ 태그 필터링 및 이진화 완료. 최종 태그 수: 4031


In [6]:
# 클래스 가중치 계산
tag_columns = mlb.classes_
tag_freq = df[tag_columns].sum()
weights = 1.0 / tag_freq
weights = weights / weights.sum() * len(tag_freq) # 정규화
weights_tensor = torch.tensor(weights.values, dtype=torch.float32)

# 가중치 저장
torch.save(weights_tensor, os.path.join(data_dir, 'tag_weights.pt'))
print("✅ 클래스 가중치 계산 및 저장 완료.")

# 데이터 분할 (80/10/10)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# CSV 저장
train_df.to_csv(os.path.join(data_dir, 'train.csv'), index=False)
val_df.to_csv(os.path.join(data_dir, 'val.csv'), index=False)
test_df.to_csv(os.path.join(data_dir, 'test.csv'), index=False)

print(f"✅ 데이터 분할 완료 - 학습: {len(train_df)}, 검증: {len(val_df)}, 테스트: {len(test_df)}")

✅ 클래스 가중치 계산 및 저장 완료.
✅ 데이터 분할 완료 - 학습: 36220, 검증: 4528, 테스트: 4528
